In [119]:
now = datetime.datetime.now()
url = f"http://openapi.seoul.go.kr:8088/{API_KEY}/json/bikeList/{2000}/{2000}/"
response = requests.get(url)
data = json.loads(response.text)
data.get('rentBikeStatus').get('row')

[{'rackTotCnt': '10',
  'stationName': '3863. 동일로58길 입구',
  'parkingBikeTotCnt': '17',
  'shared': '170',
  'stationLatitude': '37.55775452',
  'stationLongitude': '127.07297516',
  'stationId': 'ST-2543'}]

In [44]:
# 필요 라이브러리 호출
import dotenv
import os
import time
import pandas as pd
import matplotlib.pyplot as plt
import requests
import xml.etree.ElementTree as ET
import warnings
import datetime
from bs4 import BeautifulSoup
import json

# 경고창 무시
warnings.simplefilter("ignore")

dotenv.load_dotenv() # API 사용을 위한 인증키 로드
API_KEY = os.getenv('BIKE_DATA')

# 저장장소
base_output = "bike_rental_data.csv"
backup_folder = "backups_bike"

# 백업 폴더 없으면 생성
os.makedirs(backup_folder, exist_ok=True)

def save_data(data_list):
    df = pd.DataFrame(data_list)

    # 기본 데이터에 append
    if not os.path.exists(base_output):
        df.to_csv(base_output, mode='w', index=False, encoding='utf-8-sig')
    else:
        df.to_csv(base_output, mode='a', index=False, encoding='utf-8-sig', header=False)

    # 날짜별로 백업 파일 저장
    timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M')
    backup_file = os.path.join(backup_folder, f"backup_{timestamp}.csv")
    df.to_csv(backup_file, index=False, encoding='utf-8-sig')
    
# 15분 간격으로 30일간 반복
for i in range(3000):
    data_lists = []

    for s in range(1,10000,1000) :
        try:
            now = datetime.datetime.now()
            url = f"http://openapi.seoul.go.kr:8088/{API_KEY}/json/bikeList/{s}/{s+999}/"
            
            response = requests.get(url, timeout=10)
            data = json.loads(response.text)
            data_list = []
            for dt in data.get('rentBikeStatus').get('row') :
                use = {'시간' : now.strftime('%y-%m-%d %H:%M'),
                    '정류소번호': dt.get('stationName').split('.')[0],
                    '정류소이름': dt.get('stationName').split('.')[1],
                    '거치중인자전거': dt.get('parkingBikeTotCnt'),
                    '거치대개수': dt.get('rackTotCnt'),
                    }
            data_list.append(use)

        except Exception as e:
            print(f"[{s}] 수집 실패: {e}")

    data_lists.append(data_list)

    print(f"[{i+1}/8760] ✅ 저장 및 백업 완료: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    # 15분 대기
    time.sleep(450)
    print('----명령 대기중----')
    time.sleep(450)